In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core import tools
from langchain.tools import tool
from IPython.display import Markdown


load_dotenv("./.env")

llm = ChatOpenAI(model="gpt-4o")



In [2]:
@tool 
def write_email(to: str, subject:str, content: str) -> str:
    """Write and send email"""
    return f"Email sent to {to} with subject {subject} and content : {content}"

model_with_tools = llm.bind_tools( [write_email] , parallel_tool_calls=False)
output = model_with_tools.invoke("Envia un email a manuonda@gmail.com diciendo que es un crack")

print(output.tool_calls[0])
print(f"Name tool : {output.tool_calls[0]['name']}")

type(output)
args = output.tool_calls[0]['args']
print(f"Args : {args}")

#call the tool
result = write_email.invoke(args)
Markdown(result)


{'name': 'write_email', 'args': {'to': 'manuonda@gmail.com', 'subject': '¡Eres un crack!', 'content': 'Hola Manu,\n\nSolo quería decirte que eres un crack. ¡Sigue así!\n\nUn saludo,\nAnónimo'}, 'id': 'call_mjxkvIUVuUkOc8gZ6eeaKN1u', 'type': 'tool_call'}
Name tool : write_email
Args : {'to': 'manuonda@gmail.com', 'subject': '¡Eres un crack!', 'content': 'Hola Manu,\n\nSolo quería decirte que eres un crack. ¡Sigue así!\n\nUn saludo,\nAnónimo'}


Email sent to manuonda@gmail.com with subject ¡Eres un crack! and content : Hola Manu,

Solo quería decirte que eres un crack. ¡Sigue así!

Un saludo,
Anónimo

In [14]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

class StateSchema(TypedDict):
    request: str
    email: str

workflow = StateGraph(StateSchema)




def write_email_node(state: StateSchema) -> StateSchema:
    # Imperative code that processes the request
    output = model_with_tools.invoke(state["request"])
    print(output)
    if output.tool_calls:
      args = output.tool_calls[0]['args']
      email = write_email.invoke(args)
      return {"email": email}
    else:
        return {"email": output.content}



workflow = StateGraph(StateSchema)
workflow.add_node("write_email_node", write_email_node)
workflow.add_edge(START, "write_email_node")
workflow.add_edge("write_email_node", END)

app = workflow.compile()

app.invoke({"request": "Draft a response to my boss (boss@company.ai) about tomorrow's meeting"})#


content='Of course! Could you please provide more details about what you would like to convey in your response? For example, are you confirming attendance, asking for specific agenda topics, or requesting a change in time or format?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 66, 'total_tokens': 110, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'finish_reason': 'stop', 'logprobs': None} id='run-f67c36be-b38b-44af-9d02-cf624a7b97b5-0' usage_metadata={'input_tokens': 66, 'output_tokens': 44, 'total_tokens': 110, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


{'request': "Draft a response to my boss (boss@company.ai) about tomorrow's meeting",
 'email': 'Of course! Could you please provide more details about what you would like to convey in your response? For example, are you confirming attendance, asking for specific agenda topics, or requesting a change in time or format?'}

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=llm,
    tools=[write_email],
    prompt="Respond to the user's request using the tools provided"
)
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Draft a response to my boss (boss@company.ai) confirming that I want to attend Interrupt!"}]}
 )
for  m in result["messages"]:
    m.pretty_print()
    